# Analysis of borrowers' risk of default

Your project is to prepare a report for the loans division of a bank. You need to find out whether a customer's marital status and number of children have an impact on whether they will default on a loan. The bank already has some data on customers' creditworthiness.

Your report will be taken into account when creating a potential client's **credit score**. The **credit score** is used to assess a potential borrower's ability to repay their loan.

I will test whether a customer's marital status and number of children have an impact on whether they will default on a loan.

## Open the data file and look at the general information.

[Start by importing the libraries and loading the data. You may find that you need additional libraries as you go along, which is totally fine - just make sure you update this section when you do].

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/credit_scoring_eng.csv')



## Task 1. Data exploration

**Data description**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - customer's age in years
- `education` - customer's education
- `education_id` - education identifier
- `family_status` - customer's marital status
- `family_status_id` - marital status identifier
- `gender` - customer's gender
- `income_type` - type of job
- `debt` - was there any debt on the loan payment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan



In [3]:
df.shape

(21525, 12)

In [4]:
df.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family




I find it strange that the days_employed column has negative numbers, the only one that has a positive number says she is retired.
Wrong writing in educations.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



There are missing values in the days_employed and total_income columns

In [5]:
df[df['days_employed'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate




Missing days worked and total income are apparently symmetrical and mostly from people with secondary education.

In [6]:
missing_rows = df[(~df['days_employed'].isna()) & (df['total_income'].isna())]
print(missing_rows.shape[0])
missing_rows = df[(df['days_employed'].isna()) & (~df['total_income'].isna())]
print(missing_rows.shape[0])


0
0


**Intermediate conclusion**



The number of rows in the filtered table corresponds to the number of missing values, so these people without days_employed and without total income are most likely unemployed, but we can't say.

I'm going to calculate the percentage of missing values.

In [6]:
df_filtered = df[df['days_employed'].isna()]
df_filtered


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
print(df['income_type'].value_counts(normalize=True)*100)
df_filtered['income_type'].value_counts(normalize=True)*100

employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
unemployed                      0.009292
entrepreneur                    0.009292
paternity / maternity leave     0.004646
student                         0.004646
Name: income_type, dtype: float64


employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64



Students and unemployed people who do not have values for total_income, because they have no income, did not appear in the filtered table with missing values for total_income, so the bank probably did not have access to the data of these people where they appear with missing values in days_employed and total_income. in my point of view it would be more appropriate to treat this data with the median/average of the respective columns.

**Possible reasons for missing values in the data**


In addition to the above, the level of education of the apparent majority of missing values is secondary education.

I'm going to start checking whether the missing values are random.

In [7]:
print(df['education'].value_counts(normalize=True)*100)
df_filtered['education'].value_counts(normalize=True)*100

secondary education    63.879210
bachelor's degree      21.918699
SECONDARY EDUCATION     3.586527
Secondary Education     3.303136
some college            3.103368
BACHELOR'S DEGREE       1.272938
Bachelor's Degree       1.245064
primary education       1.161440
Some College            0.218351
SOME COLLEGE            0.134727
PRIMARY EDUCATION       0.078978
Primary Education       0.069686
graduate degree         0.018583
Graduate Degree         0.004646
GRADUATE DEGREE         0.004646
Name: education, dtype: float64


secondary education    64.765409
bachelor's degree      22.815087
SECONDARY EDUCATION     3.081877
Secondary Education     2.989880
some college            2.529899
Bachelor's Degree       1.149954
BACHELOR'S DEGREE       1.057958
primary education       0.873965
Some College            0.321987
SOME COLLEGE            0.321987
Primary Education       0.045998
PRIMARY EDUCATION       0.045998
Name: education, dtype: float64

**Intermediate conclusion**



However apparent the reason, the percentage of secondary_education is similar in the two data tables

In [10]:
print(df['family_status'].value_counts(normalize=True)*100)
df_filtered['family_status'].value_counts(normalize=True)*100

married              57.514518
civil partnership    19.405343
unmarried            13.068525
divorced              5.551684
widow / widower       4.459930
Name: family_status, dtype: float64


married              56.899724
civil partnership    20.331187
unmarried            13.247470
divorced              5.151794
widow / widower       4.369825
Name: family_status, dtype: float64

**Intermediate conclusion**



Just like the percentage of family_status

In [11]:
print(df['children'].value_counts(normalize=True)*100)
df_filtered['children'].value_counts(normalize=True)*100

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64


 0     66.191352
 1     21.849126
 2      9.383625
 3      1.655934
 20     0.413983
 4      0.321987
-1      0.137994
 5      0.045998
Name: children, dtype: float64

**Conclusions**



I think it's random missing values, because the percentages don't differ that much between the missing table and the normal table.
Perhaps it is some form of informal work, which is not characterized.



## Data transformation

I'm going to look at each column to see what problems we might have in them


In [12]:
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [13]:
df['education'] = df['education'].str.lower()

In [14]:
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [15]:
df['children'].value_counts(normalize=True)*100

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64



The problem is that people have -1 child and 20. I'll replace it with the median

In [16]:
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 0)

In [17]:
df['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [18]:
display((df['days_employed']<0).value_counts(normalize=True)*100)
display(df[df['days_employed']>0])

df[df['days_employed']<0] 

True     73.89547
False    26.10453
Name: days_employed, dtype: float64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21508,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21509,0,362161.054124,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21518,0,373995.710838,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [19]:

df['days_employed'] = df['days_employed'].abs()


x= (75-18)*365 #an average number of days worked at the highest age
mediana_days=df['days_employed'].median()
df.loc[df['days_employed'] > x ,'days_employed'] = mediana_days#I replaced it with the median to affect the final result as little as possible


In [20]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,2194.220567,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,2194.220567,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [21]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64



I'll replace the 0 with the median, as it would change the analysis as little as possible


In [22]:
df['dob_years'] = df['dob_years'].replace(0, int(df['dob_years'].median()))

In [23]:
df['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [24]:
df['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [25]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [26]:
df['gender'] = df['gender'].replace('XNA', 'F')
#replace with the most frequent


In [27]:
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [28]:
df['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [8]:
df.duplicated().sum()

54

In [9]:
df = df.drop_duplicates().reset_index(drop=True)

In [10]:
df.duplicated().sum()

0

In [32]:
diferenca = 21525-21454 
porcentagem_dif = (diferenca * 100)/21525
print(porcentagem_dif)
df.shape


0.32984901277584205


(21453, 12)


There were duplicates, I removed them and the dataset was reduced by 0.3% 

# Working with missing values



I'm going to work with family dictionaries and education, I think it's important

In [15]:
dicionario_educacao = df.loc[:,['education_id', 'education']]
dicionario_educacao = dicionario_educacao.drop_duplicates().reset_index(drop=True)

dicionario_familia = df.loc[:,['family_status_id', 'family_status']]
dicionario_familia = dicionario_familia.drop_duplicates().reset_index(drop=True)
display(dicionario_educacao)
dicionario_familia


,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


### Restore missing values in `total_income`


There are missing values in the 'days_employed' and 'total_income' columns. I'll replace them with the closest medians


In [34]:
def age_group_function(row):
    age = row['dob_years']
    if age <= 20:
        return 'young'
    if 21<= age <= 31:
        return 'young_adult'
    if 31 <= age <= 60:
        return 'adult'
    if age >=61:
        return 'elderly'




In [35]:
age_group_function(df.iloc[0])

'adult'

In [36]:
df['age_group'] = df.apply(age_group_function, axis=1)


/tmp/ipykernel_61/2294581768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_group'] = df.apply(age_group_function, axis=1)


In [37]:
df['age_group'].value_counts()

adult          15051
young_adult     4211
elderly         2126
young             65
Name: age_group, dtype: int64


I'm going to create functions to replace values, getting closer to the true value with a median.

In [38]:
df_sem_ausentes = df.loc[~df['total_income'].isnull()]
print(df_sem_ausentes.isna().sum())
df_sem_ausentes#without missing values

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,2194.220567,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult
21521,0,2194.220567,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,elderly
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult


In [39]:
df_sem_ausentes.pivot_table(index=['income_type','gender','education'],
              columns='age_group',
              values='total_income',
              aggfunc='mean')

age_group                                                      adult  \
income_type                 gender education                           
business                    F      bachelor's degree    36608.620325   
                                   primary education    25458.348583   
                                   secondary education  26567.907978   
                                   some college         35669.965747   
                            M      bachelor's degree    48724.012333   
                                   primary education    27392.725889   
                                   secondary education  33846.116194   
                                   some college         36988.787750   
civil servant               F      bachelor's degree    29294.233893   
                                   graduate degree      17822.757000   
                                   primary education    14339.034000   
                                   secondary education  22951.769223   
                                   some college         29495.201583   
                            M      bachelor's degree    43573.959610   
                                   primary education    39150.883000   
                                   secondary education  31219.508504   
                                   some college         41234.879571   
employee                    F      bachelor's degree    29295.902675   
                                   primary education    19370.655937   
                                   secondary education  22769.266354   
                                   some college         26772.705130   
                            M      bachelor's degree    36496.007800   
                                   graduate degree      31089.653667   
                                   primary education    22571.516268   
                                   secondary education  28105.003511   
                                   some college         35604.604154   
entrepreneur                F      bachelor's degree             NaN   
paternity / maternity leave F      secondary education   8612.661000   
retiree                     F      bachelor's degree    25361.574436   
                                   graduate degree               NaN   
                                   primary education    18351.038152   
                                   secondary education  21201.811453   
                                   some college         18463.890118   
                            M      bachelor's degree    32151.883824   
                                   graduate degree               NaN   
                                   primary education    17394.319714   
                                   secondary education  24377.182523   
                                   some college         16128.354500   
student                     M      bachelor's degree             NaN   
unemployed                  F      bachelor's degree    32435.602000   
                            M      secondary education           NaN   

age_group                                                    elderly  \
income_type                 gender education                           
business                    F      bachelor's degree    32872.322500   
                                   primary education             NaN   
                                   secondary education  29229.884122   
                                   some college         30337.601500   
                            M      bachelor's degree    46955.423550   
                                   primary education             NaN   
                                   secondary education  27338.591269   
                                   some college                  NaN   
civil servant               F      bachelor's degree    40123.141125   
                                   graduate degree               NaN   
                                   prim

In [40]:
df_pivot = df_sem_ausentes.pivot_table(index=['income_type','gender','education'],
              columns='age_group',
              values='total_income',
              aggfunc='median')
df_pivot

age_group                                                    adult  \
income_type                 gender education                         
business                    F      bachelor's degree    31358.9670   
                                   primary education    24373.9110   
                                   secondary education  23702.1650   
                                   some college         31339.3620   
                            M      bachelor's degree    39662.2450   
                                   primary education    26144.4830   
                                   secondary education  29427.8550   
                                   some college         35201.8340   
civil servant               F      bachelor's degree    25931.5100   
                                   graduate degree      17822.7570   
                                   primary education    14339.0340   
                                   secondary education  20133.0890   
                                   some college         26502.3705   
                            M      bachelor's degree    35420.7510   
                                   primary education    30545.9490   
                                   secondary education  27440.0360   
                                   some college         37303.7910   
employee                    F      bachelor's degree    25365.8830   
                                   primary education    18384.1935   
                                   secondary education  20191.4610   
                                   some college         23365.7305   
                            M      bachelor's degree    31482.2995   
                                   graduate degree      31771.3210   
                                   primary education    20991.9180   
                                   secondary education  25492.6820   
                                   some college         32693.4420   
entrepreneur                F      bachelor's degree           NaN   
paternity / maternity leave F      secondary education   8612.6610   
retiree                     F      bachelor's degree    22529.7540   
                                   graduate degree             NaN   
                                   primary education    17944.1910   
                                   secondary education  18374.8570   
                                   some college         17872.2450   
                            M      bachelor's degree    26554.6310   
                                   graduate degree             NaN   
                                   primary education    16686.8840   
                                   secondary education  22183.9670   
                                   some college         16128.3545   
student                     M      bachelor's degree           NaN   
unemployed                  F      bachelor's degree    32435.6020   
                            M      secondary education         NaN   

age_group                                                  elderly  \
income_type                 gender education                         
business                    F      bachelor's degree    29541.9080   
                                   primary education           NaN   
                                   secondary education  27706.7690   
                                   some college         30337.6015   
                            M      bachelor's degree    32235.3180   
                                   primary education           NaN   
                                   secondary education  29193.1930   
                                   some college                NaN   
civil servant               F      bachelor's degree    45649.8020   
                                   graduate degree             NaN   
                                   primary education           NaN   
                                   secondary education  23066.1870   
         



The thought process for this task here was based on having subgroups to fill in missing values from total_income in the data frame. That said, I made a pivot table with subgroups that I think affect a person's income (income type, gender and age group).


I believe in the median to avoid possible outliers

In [41]:
def aux_fill_NaN(age_group,income_type,gender,education):
    try:
        return df_pivot[age_group][income_type][gender][education]    
    except:
        return 'erro'

In [42]:
aux_fill_NaN('young_adult', 'employee', 'F', 'secondary education')


18156.236

In [43]:
df['mediana_total_income'] = df.apply(lambda row: aux_fill_NaN(row['age_group'],row['income_type'],row['gender'],row['education']), axis=1)
df['total_income']=df['total_income'].fillna(df['mediana_total_income'])

/tmp/ipykernel_61/1323637995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mediana_total_income'] = df.apply(lambda row: aux_fill_NaN(row['age_group'],row['income_type'],row['gender'],row['education']), axis=1)
/tmp/ipykernel_61/1323637995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_income']=df['total_income'].fillna(df['mediana_total_income'])


In [44]:
df[df['total_income']=='erro']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,mediana_total_income
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,erro,buy residential real estate,adult,erro


In [45]:

df['total_income'].describe()
df['total_income'] = df['total_income'].replace('erro', 23202.870000).fillna(23202.870000)
#replace with the median


/tmp/ipykernel_61/361123215.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_income'] = df['total_income'].replace('erro', 23202.870000).fillna(23202.870000)


In [46]:
print(df['total_income'].dtype)
df['total_income'].isnull().sum()


float64


0

###  Restore values in `days_employed`

In [47]:

dfp = df_sem_ausentes.pivot_table(index=['income_type','gender','education'],
              columns='age_group',
              values='days_employed',
              aggfunc='median')
dfp


age_group                                                     adult  \
income_type                 gender education                          
business                    F      bachelor's degree    1793.422153   
                                   primary education    1166.810199   
                                   secondary education  1875.972013   
                                   some college         1355.477051   
                            M      bachelor's degree    1604.049656   
                                   primary education    2417.631813   
                                   secondary education  1849.622944   
                                   some college         1536.232135   
civil servant               F      bachelor's degree    3135.355524   
                                   graduate degree      5968.075884   
                                   primary education    2787.767403   
                                   secondary education  3209.900710   
                                   some college         2168.367551   
                            M      bachelor's degree    3746.419221   
                                   primary education    4193.718465   
                                   secondary education  3056.595550   
                                   some college         3551.609375   
employee                    F      bachelor's degree    1998.195762   
                                   primary education    1534.423060   
                                   secondary education  2039.246503   
                                   some college         1626.414565   
                            M      bachelor's degree    1567.431757   
                                   graduate degree      2351.431934   
                                   primary education     971.256328   
                                   secondary education  1605.639152   
                                   some college         1423.316912   
entrepreneur                F      bachelor's degree            NaN   
paternity / maternity leave F      secondary education  3296.759962   
retiree                     F      bachelor's degree    2194.220567   
                                   graduate degree              NaN   
                                   primary education    2194.220567   
                                   secondary education  2194.220567   
                                   some college         2194.220567   
                            M      bachelor's degree    2194.220567   
                                   graduate degree              NaN   
                                   primary education    2194.220567   
                                   secondary education  2194.220567   
                                   some college         2194.220567   
student                     M      bachelor's degree            NaN   
unemployed                  F      bachelor's degree    2194.220567   
                            M      secondary education          NaN   

age_group                                                   elderly  \
income_type                 gender education                          
business                    F      bachelor's degree    3211.875664   
                                   primary education            NaN   
                                   secondary education  2930.866014   
                                   some college          927.335994   
                            M      bachelor's degree    2233.174531   
                                   primary education            NaN   
                                   secondary education  1804.340536   
                                   some college                 NaN   
civil servant               F      bachelor's degree    5452.719249   
                                   graduate degree              NaN   
                                   primary education            NaN   
                       

In [48]:
df_sem_ausentes.pivot_table(index=['income_type','gender','education'],
              columns='age_group',
              values='days_employed',
              aggfunc='mean')

age_group                                                     adult  \
income_type                 gender education                          
business                    F      bachelor's degree    2343.726651   
                                   primary education    1698.213985   
                                   secondary education  2480.150750   
                                   some college         1873.009192   
                            M      bachelor's degree    2045.389668   
                                   primary education    3401.776086   
                                   secondary education  2384.206752   
                                   some college         1926.661756   
civil servant               F      bachelor's degree    3602.999948   
                                   graduate degree      5968.075884   
                                   primary education    2787.767403   
                                   secondary education  4151.137421   
                                   some college         2634.927178   
                            M      bachelor's degree    3972.645207   
                                   primary education    4855.517432   
                                   secondary education  3417.895924   
                                   some college         4272.903958   
employee                    F      bachelor's degree    2813.554634   
                                   primary education    2274.105996   
                                   secondary education  2838.203238   
                                   some college         1972.846951   
                            M      bachelor's degree    2253.330358   
                                   graduate degree      2704.223421   
                                   primary education    1460.943610   
                                   secondary education  2303.879398   
                                   some college         1978.164710   
entrepreneur                F      bachelor's degree            NaN   
paternity / maternity leave F      secondary education  3296.759962   
retiree                     F      bachelor's degree    2194.220567   
                                   graduate degree              NaN   
                                   primary education    2194.220567   
                                   secondary education  2194.220567   
                                   some college         2194.220567   
                            M      bachelor's degree    2194.220567   
                                   graduate degree              NaN   
                                   primary education    2194.220567   
                                   secondary education  2194.220567   
                                   some college         2194.220567   
student                     M      bachelor's degree            NaN   
unemployed                  F      bachelor's degree    2194.220567   
                            M      secondary education          NaN   

age_group                                                   elderly  \
income_type                 gender education                          
business                    F      bachelor's degree    4720.056765   
                                   primary education            NaN   
                                   secondary education  4046.248926   
                                   some college          927.335994   
                            M      bachelor's degree    3763.411334   
                                   primary education            NaN   
                                   secondary education  2153.011473   
                                   some college                 NaN   
civil servant               F      bachelor's degree    7045.752440   
                                   graduate degree              NaN   
                                   primary education            NaN   
                       



I'll use median because of the outliers

In [49]:
def aux_fill_NaN_day(age_group,income_type,gender,education):
    try:
        return df_pivot[age_group][income_type][gender][education]    
    except:
        return 'error'

In [50]:

aux_fill_NaN_day('young_adult', 'employee', 'F', 'secondary education')


18156.236

In [51]:

df['mediana_days_employed'] = df.apply(lambda row: aux_fill_NaN_day(row['age_group'],row['income_type'],row['gender'],row['education']), axis=1)
df['days_employed']=df['days_employed'].fillna(df['mediana_days_employed'])



/tmp/ipykernel_61/2334890380.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mediana_days_employed'] = df.apply(lambda row: aux_fill_NaN_day(row['age_group'],row['income_type'],row['gender'],row['education']), axis=1)
/tmp/ipykernel_61/2334890380.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_employed']=df['days_employed'].fillna(df['mediana_days_employed'])


In [52]:

df['days_employed'].isnull().sum()

2

In [53]:
df['days_employed'].median
df['days_employed'] = df['days_employed'].fillna(8437.673028)



/tmp/ipykernel_61/2517550483.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_employed'] = df['days_employed'].fillna(8437.673028)


In [54]:
print(df.shape)
df['total_income']


(21453, 15)


0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21453, dtype: float64

## Categorizing data

I will categorize based on income


In [55]:
df['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21453, dtype: float64

In [56]:
print(df['total_income'].unique)
df['total_income'].describe()

<bound method Series.unique of 0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 21453, dtype: float64>


count     21453.000000
mean      26478.599651
std       15750.945526
min        3306.762000
25%       17191.455000
50%       23222.083000
75%       31535.633000
max      362496.645000
Name: total_income, dtype: float64

I'll categorize them into 4 groups, low, average, high and super_high

In [57]:
def total_income_function(row):
    income = row['total_income']
    if income < 17191.455000:
        return 'low'
    if 17191.455900<= income <= 23223.899000:
        return 'average'
    if 23223.899900 <= income <= 31536.563000:
        return 'high'
    if income >=31536.563900:
        return 'super_high'
    #values based on describe

In [58]:
df['group_income'] = df.apply(total_income_function, axis=1)

total_income_function(df.iloc[0])


/tmp/ipykernel_61/1152470786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['group_income'] = df.apply(total_income_function, axis=1)


'super_high'

In [59]:
df['group_income'].value_counts()

average       5364
low           5363
high          5363
super_high    5362
Name: group_income, dtype: int64

In [60]:
df['group_income'].describe()

count       21452
unique          4
top       average
freq         5364
Name: group_income, dtype: object

## Checking the Hypotheses


**Is there a correlation between the level of income and paying on time?**

In [61]:

dfc= df.pivot_table(index=['children', 'debt'], values='total_income', aggfunc='count').reset_index()
display(dfc)
taxalist = [
((dfc['total_income'][1])*100) / (dfc['total_income'][0]+dfc['total_income'][1]),
    ((dfc['total_income'][3])*100) / (dfc['total_income'][2]+dfc['total_income'][3]),
    ((dfc['total_income'][5])*100) / (dfc['total_income'][4]+dfc['total_income'][5]),
    ((dfc['total_income'][7])*100) / (dfc['total_income'][6]+dfc['total_income'][7]),
    ((dfc['total_income'][9])*100) / (dfc['total_income'][8]+dfc['total_income'][9]),
      
]
display(taxalist)


,children,debt,total_income
0,0,0,13141
1,0,1,1072
2,1,0,4364
3,1,1,444
4,2,0,1858
5,2,1,194
6,3,0,303
7,3,1,27
8,4,0,37
9,4,1,4


[7.542390769014283,
 9.234608985024959,
 9.454191033138402,
 8.181818181818182,
 9.75609756097561]

In [69]:
inadimplencia_filhos = df.pivot_table(index='children', values='debt', aggfunc='mean')
display(inadimplencia_filhos)

,debt
children,
0,0.075424
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000


**Conclusion**



Having children does not seem to be a factor that increases the default rate, because even though it increased by 2% from 0 children to 1, it is not much and with 3 children the rate drops, becoming similar to 0 children. 


**Is there a correlation between family status and paying on time?**

In [62]:

dfc2= df.pivot_table(index=['family_status', 'debt'], values='total_income', aggfunc='count').reset_index()
display(dfc2)
taxalist2=[
((dfc2['total_income'][1])*100) / (dfc2['total_income'][0] + dfc2['total_income'][1]), 
((dfc2['total_income'][3])*100) / (dfc2['total_income'][2]+dfc2['total_income'][3]), 
(dfc2['total_income'][5])*100 / (dfc2['total_income'][4]+dfc2['total_income'][5]), 
(dfc2['total_income'][7])*100 / (dfc2['total_income'][6]+dfc2['total_income'][7]),
(dfc2['total_income'][9])*100 / (dfc2['total_income'][8]+dfc2['total_income'][9]),
]
taxalist2


,family_status,debt,total_income
0,civil partnership,0,3762
1,civil partnership,1,388
2,divorced,0,1110
3,divorced,1,85
4,married,0,11408
5,married,1,931
6,unmarried,0,2536
7,unmarried,1,274
8,widow / widower,0,896
9,widow / widower,1,63


[9.349397590361447,
 7.112970711297071,
 7.5451819434313965,
 9.750889679715302,
 6.569343065693431]

**Conclusion**



It seems that people who have not been married to another person are more likely to default, even if they have a civil partnership.

**Is there a correlation between family status and paying on time?**

In [63]:

dfc3= df.pivot_table(index=['group_income', 'debt'], values='total_income', aggfunc='count').reset_index()
display(dfc3)


taxalist3 = [(dfc3['total_income'][1])*100 / (dfc3['total_income'][0]+dfc3['total_income'][1]),
             (dfc3['total_income'][3])*100 / (dfc3['total_income'][2]+dfc3['total_income'][3]),
             (dfc3['total_income'][5])*100 / (dfc3['total_income'][4]+dfc3['total_income'][5]),
             (dfc3['total_income'][7])*100 / (dfc3['total_income'][6]+dfc3['total_income'][7]),
            ]
taxalist3


,group_income,debt,total_income
0,average,0,4897
1,average,1,467
2,high,0,4894
3,high,1,469
4,low,0,4937
5,low,1,426
6,super_high,0,4983
7,super_high,1,379


[8.706189410887397, 8.74510535148238, 7.943315308595935, 7.068258112644536]

**Conclusion**



 The person's income group doesn't seem to have much influence either, but the richest have the lowest rate, followed by the poorest. This makes the 'middle' groups the most likely to default. 

**How does the purpose of credit affect the default rate?**

In [64]:
dfc4= df.pivot_table(index=['purpose', 'debt'], values='total_income', aggfunc='count').reset_index()
dfc4


,purpose,debt,total_income
0,building a property,0,565
1,building a property,1,54
2,building a real estate,0,576
3,building a real estate,1,48
4,buy commercial real estate,0,614
...,...,...,...
71,transactions with my real estate,1,50
72,university education,0,412
73,university education,1,40
74,wedding ceremony,0,727


**Conclusion**



The reason for the credit is not conclusive, but something personal.


# General conclusion 




Wealthier people who have been through or are in a marriage with no children or more than two tend to pay back more of their loans 